## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [0]:
import os
import tensorflow as tf
import keras
import numpy as np
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Dropout
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.utils import to_categorical

In [16]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
x_train = x_train/255
x_test = x_test/255

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
## 超參數設定
lr = 0.01
EPOCHS = 20
BATCH_SIZE = 256

In [27]:
# Import ResNet50 as basis model 
model_b = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(32,32,3))

# Abandon the FC-layer on the top of ResNet50
x = model_b.output
x = Flatten()(x)
#x = Dropout(0.5)(x)
output_layer = Dense(num_classes, activation = 'softmax')(x)
model = Model(inputs=model_b.input, outputs=output_layer)
model.summary()


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
# Compile

model.compile(optimizer=Adam(lr=lr),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE,
                              epochs=EPOCHS,                              
                              verbose=1,
                              validation_data=(x_test, y_test))

scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 22s 444us/step - loss: 1.6385 - acc: 0.4206 - val_loss: 2.1514 - val_acc: 0.3451
Epoch 2/20
50000/50000 [==============================] - 22s 444us/step - loss: 1.6356 - acc: 0.4186 - val_loss: 8.6127 - val_acc: 0.1224
Epoch 3/20
50000/50000 [==============================] - 22s 442us/step - loss: 1.6220 - acc: 0.4224 - val_loss: 2.1740 - val_acc: 0.2742
Epoch 4/20
50000/50000 [==============================] - 22s 441us/step - loss: 1.6250 - acc: 0.4258 - val_loss: 7.9724 - val_acc: 0.1464
Epoch 5/20
50000/50000 [==============================] - 22s 443us/step - loss: 1.5425 - acc: 0.4403 - val_loss: 1.6283 - val_acc: 0.4171
Epoch 6/20
50000/50000 [==============================] - 22s 444us/step - loss: 1.4957 - acc: 0.4694 - val_loss: 5.3759 - val_acc: 0.2015
Epoch 7/20
50000/50000 [==============================] - 22s 444us/step - loss: 1.8190 - acc: 0.3741 - val_loss: 2.